The following code is written with this tutoral as an inspiration: https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [1]:
import dill
dill.dump_session('Diamonds_XGBoost_Validation.kernel')

In [24]:
import dill
dill.load_session('Diamonds_XGBoost_Validation.kernel')

c:\it\machine learning course\mlenv\lib\site-packages\dill\_dill.py:472: MatplotlibDeprecationWarning: 
The examples.directory rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2. In the future, examples will be found relative to the 'datapath' directory.
  obj = StockUnpickler.load(self)
c:\it\machine learning course\mlenv\lib\site-packages\dill\_dill.py:472: MatplotlibDeprecationWarning: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  obj = StockUnpickler.load(self)
c:\it\machine learning course\mlenv\lib\site-packages\dill\_dill.py:472: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  obj = StockUnpickler.load(self)
c:\it\machine learning course\mlenv\lib\site-packages\dill\_dill.py:472: MatplotlibDeprecationWarning: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
  obj = StockUnpickler.load(self)
c:\it\machin

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import cross_validate
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [4]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams["figure.figsize"] = 12, 4

In [5]:
pp_1_train = pd.read_csv("diamonds_tr_80_pp_1.csv")
pp_1_test = pd.read_csv("diamonds_tr_20_pp_1.csv")
                        
pp_2_train = pd.read_csv("diamonds_tr_80_pp_2.csv")
pp_2_test = pd.read_csv("diamonds_tr_20_pp_2.csv")

pca_train = pd.read_csv("diamonds_tr_80_pca.csv")
pca_test = pd.read_csv("diamonds_tr_20_pca.csv")

In [6]:
target = "price"

In [7]:
def modelfit(alg, dtrain, dtest, predictors, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='mae', early_stopping_rounds=early_stopping_rounds, verbose_eval=False)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['price'], eval_metric='mae')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtest_predictions = alg.predict(dtest[predictors])
    #dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Train MAE : %.4g" % metrics.mean_absolute_error(dtrain['price'].values, dtrain_predictions))
    print ("Test MAE : %.4g" % metrics.mean_absolute_error(dtest['price'].values, dtest_predictions))
    print("Best Iteration: {}".format(alg.get_booster().best_iteration))
    print(cvresult.shape[0])
    #print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['price'], dtrain_predprob))
                    
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #feat_imp.plot(kind='bar', title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

In [8]:
predictors = [x for x in pp_1_train.columns if x not in [target]]
predictors

['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']

### Test the best dataset version

In [9]:
xgb1 = XGBRegressor(
    learning_rate=0.1,
    n_estimators=1000,
    max_depth=5,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    objective="reg:squarederror",
    nthread=4,
    scale_pos_weight=1,
    seed=27
    #,tree_method='gpu_hist',gpu_id=0
    )

modelfit(xgb1, pp_1_train, pp_1_test, predictors)


Model Report
Train MAE : 213.7
Test MAE : 274.9
Best Iteration: 444
445


In [10]:
modelfit(xgb1, pp_2_train, pp_2_test, predictors)


Model Report
Train MAE : 214.1
Test MAE : 274.5
Best Iteration: 444
445


In [11]:
pca_predictors = [x for x in pca_train.columns if x not in [target]]
modelfit(xgb1, pca_train, pca_test, pca_predictors)


Model Report
Train MAE : 251.7
Test MAE : 325
Best Iteration: 444
445


pp_2 is best by a slight margin

### Step 2: Tune max_depth and min_child_weight

In [12]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}

In [13]:
gsearch1 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=445, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='neg_mean_absolute_error',n_jobs=4,iid=False, cv=5)
gsearch1.fit(pp_2_train[predictors],pp_2_train[target])

c:\it\machine learning course\mlenv\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=0.8, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=5,
                                    min_child_weight=1, missing=None,
                                    n_estimators=445, n_jobs=1, nthread=4,
                                    objective='reg:squarederror',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=27, silent=None,
                                    subsample=0.8, verbosity=1),
             iid=False, n_jobs=4,
             param_grid={'max_depth': range(3, 10, 2),
                         'min_child_weight': range(1, 6, 2)},
             pre_dis

In [14]:
gsearch1.best_params_, gsearch1.best_score_

({'max_depth': 9, 'min_child_weight': 1}, -273.1912654232095)

In [15]:
param_test2 = {
 'max_depth':[8,9,10],
 'min_child_weight':[1,2,3]
}

In [4]:
gsearch2 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=445, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test2, scoring='neg_mean_absolute_error',n_jobs=4,iid=False, cv=5)
gsearch2.fit(pp_2_train[predictors],pp_2_train[target])

c:\it\machine learning course\mlenv\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=0.8, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=5,
                                    min_child_weight=1, missing=None,
                                    n_estimators=445, n_jobs=1, nthread=4,
                                    objective='reg:squarederror',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=27, silent=None,
                                    subsample=0.8, verbosity=1),
             iid=False, n_jobs=4,
             param_grid={'max_depth': [8, 9, 10],
                         'min_child_weight': [1, 2, 3]},
             pre_dispatch='2*n

In [5]:
gsearch2.best_params_, gsearch2.best_score_

({'max_depth': 9, 'min_child_weight': 1}, -273.1912654232095)

### Tune gamma

In [6]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}

In [7]:
gsearch3 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=445, max_depth=9,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring='neg_mean_absolute_error',n_jobs=4,iid=False, cv=5)
gsearch3.fit(pp_2_train[predictors],pp_2_train[target])
gsearch3.best_params_, gsearch3.best_score_

({'gamma': 0.1}, -273.1610849721122)

In [8]:
xgb2 = XGBRegressor(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=9,
 min_child_weight=1,
 gamma=0.1,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'reg:squarederror',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb2, pp_2_train, pp_2_test, predictors)


Model Report
Train MAE : 178.5
Test MAE : 267.3
Best Iteration: 95
96


### Step 4: Tune subsample and colsample_bytree

In [9]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

In [10]:
gsearch4 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=96, max_depth=9,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test4, scoring='neg_mean_absolute_error',n_jobs=4,iid=False, cv=5)
gsearch4.fit(pp_2_train[predictors],pp_2_train[target])
gsearch4.best_params_, gsearch4.best_score_

c:\it\machine learning course\mlenv\lib\site-packages\sklearn\model_selection\_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


({'colsample_bytree': 0.9, 'subsample': 0.9}, -270.2511072785668)

In [11]:
param_test5 = {
 'subsample':[i/100.0 for i in range(85,100,5)],
 'colsample_bytree':[i/100.0 for i in range(85,100,5)]
}

In [12]:
gsearch5 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=96, max_depth=9,
 min_child_weight=1, gamma=0.1, subsample=0.8, colsample_bytree=0.8,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test5, scoring='neg_mean_absolute_error',n_jobs=4,iid=False, cv=5)
gsearch5.fit(pp_2_train[predictors],pp_2_train[target])
gsearch5.best_params_, gsearch5.best_score_

({'colsample_bytree': 0.9, 'subsample': 0.9}, -270.2511072785668)

### Step 5: Tuning Regularization Parameters

In [13]:
param_test6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

In [14]:
gsearch6 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=96, max_depth=9,
 min_child_weight=1, gamma=0.1, subsample=0.9, colsample_bytree=0.9,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test6, scoring='neg_mean_absolute_error',n_jobs=4,iid=False, cv=5)
gsearch6.fit(pp_2_train[predictors],pp_2_train[target])
gsearch6.best_params_, gsearch6.best_score_

({'reg_alpha': 1}, -270.19037130997106)

In [15]:
param_test7 = {
 'reg_alpha':[0.5, 0.7, 1, 1.5, 5]
}

In [17]:
gsearch7 = GridSearchCV(estimator = XGBRegressor( learning_rate =0.1, n_estimators=96, max_depth=9,
 min_child_weight=1, gamma=0.1, subsample=0.9, colsample_bytree=0.9,
 objective= 'reg:squarederror', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test7, scoring='neg_mean_absolute_error',n_jobs=4,iid=False, cv=5)
gsearch7.fit(pp_2_train[predictors],pp_2_train[target])
gsearch7.best_params_, gsearch7.best_score_

({'reg_alpha': 1}, -270.19037130997106)

In [18]:
xgb3 = XGBRegressor(
     learning_rate =0.1,
     n_estimators=1000,
     max_depth=9,
     min_child_weight=1,
     gamma=0.1,
     subsample=0.9,
     colsample_bytree=0.9,
        reg_alpha=1,
     objective= 'reg:squarederror',
     nthread=4,
     scale_pos_weight=1,
     seed=27)
modelfit(xgb3, pp_2_train, pp_2_test, predictors)


Model Report
Train MAE : 170.6
Test MAE : 265.5
Best Iteration: 104
105


### Step 6: Reducing Learning Rate

In [20]:
xgb4 = XGBRegressor(
     learning_rate =0.01,
     n_estimators=5000,
     max_depth=9,
     min_child_weight=1,
     gamma=0.1,
     subsample=0.9,
     colsample_bytree=0.9,
        reg_alpha=1,
     objective= 'reg:squarederror',
     nthread=4,
     scale_pos_weight=1,
     seed=27)
modelfit(xgb4, pp_2_train, pp_2_test, predictors)


Model Report
Train MAE : 154
Test MAE : 262.3
Best Iteration: 1421
1422


In [22]:
validation = pd.read_csv("diamonds_val_pp_2.csv")
training = pd.read_csv("diamonds_tr_pp_2.csv")
len(validation), len(training)

(10787, 43153)

In [23]:
modelfit(xgb4, training, validation, predictors)


Model Report
Train MAE : 161.8
Test MAE : 256.6
Best Iteration: 1421
1422
